In [ ]:
import sys
sys.path.append('../../')

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Concatenate,Bidirectional, Attention
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# Load and preprocess data
def load_data(file_path):
    df = pd.read_csv(file_path, header=None, names=['event_type', 'agent_id', 'context'])
    return df

def preprocess_data(df):
    le_event = LabelEncoder()
    le_agent = LabelEncoder()
    le_context = LabelEncoder()

    # Replace empty strings with a special token
    df['context'] = df['context'].replace('', '<EMPTY>')

    df['event_type_encoded'] = le_event.fit_transform(df['event_type'])
    df['agent_id_encoded'] = le_agent.fit_transform(df['agent_id'])
    df['context_encoded'] = le_context.fit_transform(df['context'])

    return df, le_event, le_agent, le_context


In [4]:
# Create sequences
def create_sequences(df, sequence_length):
    sequences = []
    targets = []

    for i in range(len(df) - sequence_length):
        seq = df.iloc[i:i+sequence_length]
        target = df.iloc[i+sequence_length]

        sequences.append(seq[['event_type_encoded', 'agent_id_encoded', 'context_encoded']].values)
        targets.append([target['event_type_encoded'], target['agent_id_encoded'], target['context_encoded']])

    return np.array(sequences), np.array(targets)

In [5]:
# Build model
def build_model(input_shape, num_classes):
    input_event = Input(shape=(input_shape[0], 1))
    input_agent = Input(shape=(input_shape[0], 1))
    input_context = Input(shape=(input_shape[0], 1))

    concat = Concatenate()([input_event, input_agent, input_context])

    lstm = LSTM(64, return_sequences=False)(concat)

    output_event = Dense(num_classes[0], activation='softmax', name='event_type')(lstm)
    output_agent = Dense(num_classes[1], activation='softmax', name='agent_id')(lstm)
    output_context = Dense(num_classes[2], activation='softmax', name='context')(lstm)

    model = Model(inputs=[input_event, input_agent, input_context],
                  outputs=[output_event, output_agent, output_context])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy','accuracy','accuracy'])

    return model

In [6]:
def predict_next_moves(model, initial_sequence, le_event, le_agent, le_context, n_predictions):
    predictions = []
    current_sequence = initial_sequence.copy()

    for _ in range(n_predictions):
        custom_event = current_sequence[:, 0].reshape(1, -1, 1)
        custom_agent = current_sequence[:, 1].reshape(1, -1, 1)
        custom_context = current_sequence[:, 2].reshape(1, -1, 1)

        prediction = model.predict([custom_event, custom_agent, custom_context])

        predicted_event = le_event.inverse_transform([np.argmax(prediction[0])])[0]
        predicted_agent = le_agent.inverse_transform([np.argmax(prediction[1])])[0]
        predicted_context = le_context.inverse_transform([np.argmax(prediction[2])])[0]

        predicted_context = '' if predicted_context == '<EMPTY>' else predicted_context

        predictions.append((predicted_event, predicted_agent, predicted_context))

        # Update the sequence for the next prediction
        new_row = np.array([[
            le_event.transform([predicted_event])[0],
            le_agent.transform([predicted_agent])[0],
            le_context.transform([predicted_context if predicted_context != '' else '<EMPTY>'])[0]
        ]])
        current_sequence = np.vstack((current_sequence[1:], new_row))

    return predictions

In [7]:
# Load and preprocess data
df = load_data('..\..\data\processed/games/tic-tac-toe/1k_single_agent.csv')
df, le_event, le_agent, le_context = preprocess_data(df)

# Create sequences
sequence_length = 10  # Changed to 10
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train_event = X_train[:, :, 0].reshape(X_train.shape[0], X_train.shape[1], 1)
X_train_agent = X_train[:, :, 1].reshape(X_train.shape[0], X_train.shape[1], 1)
X_train_context = X_train[:, :, 2].reshape(X_train.shape[0], X_train.shape[1], 1)

X_test_event = X_test[:, :, 0].reshape(X_test.shape[0], X_test.shape[1], 1)
X_test_agent = X_test[:, :, 1].reshape(X_test.shape[0], X_test.shape[1], 1)
X_test_context = X_test[:, :, 2].reshape(X_test.shape[0], X_test.shape[1], 1)

y_train_event = to_categorical(y_train[:, 0], num_classes=len(le_event.classes_))
y_train_agent = to_categorical(y_train[:, 1], num_classes=len(le_agent.classes_))
y_train_context = to_categorical(y_train[:, 2], num_classes=len(le_context.classes_))

y_test_event = to_categorical(y_test[:, 0], num_classes=len(le_event.classes_))
y_test_agent = to_categorical(y_test[:, 1], num_classes=len(le_agent.classes_))
y_test_context = to_categorical(y_test[:, 2], num_classes=len(le_context.classes_))

# Build and train model
model = build_model(X_train_event.shape[1:],
                    [len(le_event.classes_), len(le_agent.classes_), len(le_context.classes_)])

model.fit([X_train_event, X_train_agent, X_train_context],
            [y_train_event, y_train_agent, y_train_context],
            validation_data=([X_test_event, X_test_agent, X_test_context],
                            [y_test_event, y_test_agent, y_test_context]),
            epochs=50, batch_size=32)

# Evaluate model
results = model.evaluate(
        [X_test_event, X_test_agent, X_test_context],
        [y_test_event, y_test_agent, y_test_context]
    )
    
    # Print evaluation results
print("Test Results:")
for metric_name, value in zip(model.metrics_names, results):
        print(f"{metric_name}: {value:.4f}")




Epoch 1/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - agent_id_accuracy: 0.4710 - context_accuracy: 0.1311 - event_type_accuracy: 0.7884 - loss: 4.1732 - val_agent_id_accuracy: 0.7447 - val_context_accuracy: 0.2149 - val_event_type_accuracy: 0.9316 - val_loss: 2.9685
Epoch 2/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - agent_id_accuracy: 0.8371 - context_accuracy: 0.2389 - event_type_accuracy: 0.9283 - loss: 2.8035 - val_agent_id_accuracy: 0.9342 - val_context_accuracy: 0.2651 - val_event_type_accuracy: 0.9342 - val_loss: 2.3603
Epoch 3/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - agent_id_accuracy: 0.9265 - context_accuracy: 0.2512 - event_type_accuracy: 0.9311 - loss: 2.3485 - val_agent_id_accuracy: 0.9342 - val_context_accuracy: 0.2651 - val_event_type_accuracy: 0.9337 - val_loss: 2.2608
Epoch 4/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - agent_id_accuracy: 0.9353 - context_accuracy: 0.2611 - event_type_accuracy: 0.9345 - loss: 2.2728 - val_agent_id_accuracy: 0.9368 - val_contex

In [9]:
# Predict next moves for a random sequence from the dataset
import random
k = 10  # size of initial sequence, changed to 10
n_predictions = 15  # number of moves to predict

# Select a random sequence from the dataset
random_index = random.randint(0, len(X) - 1)
initial_sequence = X[random_index]

predictions = predict_next_moves(model, initial_sequence, le_event, le_agent, le_context, n_predictions)

print("Initial sequence:")
for i in range(k):
    event = le_event.inverse_transform([initial_sequence[i, 0]])[0]
    agent = le_agent.inverse_transform([initial_sequence[i, 1]])[0]
    context = le_context.inverse_transform([initial_sequence[i, 2]])[0]
    context = '' if context == '<EMPTY>' else context
    print(f"{event}, {agent}, {context}")

print("\nPredicted moves:")
for i, (event, agent, context) in enumerate(predictions, 1):
    print(f"Move {i}: {event}, {agent}, {context}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Initial sequence:
MOVE, X, 1,2
MOVE, O, 0,0
MOVE, X, 0,1
MOVE, O, 2,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 1,0
MOVE, X, 2,2
MOVE, O, 2,1

Predicted moves:
Move 1: MOVE, X, 0,0
Move 2: MOVE, O, 0,2
Move 3: MOVE, X, 1,2
Move 4: MOVE, O, X
Move 5: MOVE, X, 0,2
Move 6: GAME_END, system, X
Move 7: GAME_START, system, New Game
Move 8: MOVE, X, 2,0
Move 9: MOVE, O, 0,2
Move 10: MOVE, X, 0,0
Move 11:

In [10]:
# Assume `model` is your Keras model
model.save('../../results/models/RNN_LSTM.h5')

In [13]:
# custom_log = np.array([
#         [le_event.transform(['GAME_START'])[0], le_agent.transform(['system'])[0], le_context.transform(['New Game'])[0]],
#         [le_event.transform(['MOVE'])[0], le_agent.transform(['X'])[0], le_context.transform(['2,2'])[0]],
#         [le_event.transform(['MOVE'])[0], le_agent.transform(['O'])[0], le_context.transform(['2,0'])[0]]
#     ])

# # Pad the custom log to match the sequence length of 10


# for _ in range(5):
#     # Pad the custom log to match the sequence length of 10
#     if custom_log.shape[0] < sequence_length:
#         padding = np.zeros((sequence_length - custom_log.shape[0], 3), dtype=int)
#         custom_log_padded = np.vstack((padding, custom_log))
#     else:
#         custom_log_padded = custom_log[-sequence_length:]  # Keep only the last sequence_length elements

#     custom_event = custom_log_padded[:, 0].reshape(1, sequence_length, 1)
#     custom_agent = custom_log_padded[:, 1].reshape(1, sequence_length, 1)
#     custom_context = custom_log_padded[:, 2].reshape(1, sequence_length, 1)

#     predictions = model.predict([custom_event, custom_agent, custom_context])

#     predicted_event = le_event.inverse_transform([np.argmax(predictions[0])])
#     predicted_agent = le_agent.inverse_transform([np.argmax(predictions[1])])
#     predicted_context = le_context.inverse_transform([np.argmax(predictions[2])])

#     print(f"Predicted next move: {predicted_event[0]}, {predicted_agent[0]}, {predicted_context[0]}")
#     custom_log = np.vstack((custom_log, [le_event.transform([predicted_event[0]])[0], le_agent.transform([predicted_agent[0]])[0], le_context.transform([predicted_context[0]])[0]]))
    